# Reinforcement Tasks

En el problemas de aprendizaje, a diferencia de en programación dinámica, no conocemos los detalles del problema. Por lo tanto vamos a modelar un problema de aprendizaje con dos "clases". 

1. La primera clase va a definir la tarea. Vamos a nombrar a esta tarea RLEnvironment
2. La segunda clase define al agente que aprende a resolver la tarea maximizando su ganancia.

Vamos a ver como construir cada una de estas clases de manera que tengamos un cascarón para distintas tareas de aprendizaje.


## Una tarea de ejemplo: el problema el viajero

Para poder entender como modelar un problema con estas clases. Vamos a programar un agente que aprenda a encontrar un camino optimo para visitar todas las ciudades de Europa empezando desde cualquier ciudad inicial.

La informacion de distancia entre ciudades esta dada en la siguiente tabla

In [1]:
dist_info = readcsv("data/DistMat.csv")

25×25 Array{Any,2}:
 "Origin"                "Barcelona"  …      "Vienna"      "Warsaw"
 "Barcelona"            0                1347.43       1862.33     
 "Belgrade"          1528.13              489.28        826.66     
 "Berlin"            1497.61              523.61        516.06     
 "Brussels"          1062.89              914.81       1159.85     
 "Bucharest"         1968.42          …   855.32        946.12     
 "Budapest"          1498.79              216.98        545.29     
 "Copenhagen"        1757.54              868.87        667.8      
 "Dublin"            1469.29             1680          1823.72     
 "Hamburg"           1471.78              742.79        750.49     
 "Istanbul"          2230.42          …  1273.88       1386.08     
 "Kiev"              2391.06             1052.76        690.12     
 "London"            1137.67             1233.48       1445.85     
 "Madrid"             504.64             1807.09       2288.42     
 "Milan"              725.12

Una de las estrucutras basicas de Julia son los NamedArrays, que funcionan de manera similar a las matrices con nombres de dimensiones en R. Usaremos NamedArrays para guardar la informacion de distancias de manera que sea facil de accesar. Para poder usar NamedArrays necesitamos usar una paqueteria adicional a julia base con el comando:

In [2]:
# Pkg.add("NamedArrays") # para instalar por primera vez
using NamedArrays

A continuacion creamos un named array con la informacion de distancias y nombres dimenciones las ciudades

In [3]:
cities = dist_info[2:end, 1] 
distances = dist_info[2:end, 2:end]
distArray = NamedArray(distances, (cities, cities))

24×24 Named Array{Any,2}
           A ╲ B │        Barcelona  …            Warsaw
─────────────────┼──────────────────────────────────────
Barcelona        │                0  …           1862.33
Belgrade         │          1528.13               826.66
Berlin           │          1497.61               516.06
Brussels         │          1062.89              1159.85
Bucharest        │          1968.42               946.12
Budapest         │          1498.79               545.29
Copenhagen       │          1757.54                667.8
Dublin           │          1469.29              1823.72
Hamburg          │          1471.78               750.49
Istanbul         │          2230.42              1386.08
Kiev             │          2391.06               690.12
London           │          1137.67              1445.85
Madrid           │           504.64              2288.42
Milan            │           725.12              1143.01
Moscow           │          3006.93              1149.41
Munich

In [4]:
d = distArray["Barcelona", "Warsaw"]
println("Distance from Barcelona to Warsaw: $d km")

Distance from Barcelona to Warsaw: 1862.33 km


Hubieramos podido implementar algo similar usando diccionarios, que son un tipo nativo de estructura de datos de julia y muchos otros lenguages. Pongo aqui la implementacion solo por curiosidad, aunque usaremos `distArray` para todos los calculos futuros.

In [5]:
distDict = Dict([([cities[i], cities[j]], distances[i, j]) for i in 1:24, j in 1:24])
d = distDict[["Barcelona", "Warsaw"]] # notemos los brackets adicionales
println("Distance from Barcelona to Warsaw: $d km")

Distance from Barcelona to Warsaw: 1862.33 km


## Reinforcement Learning Environments

Los elementos que debe tener una tarea de aprendizaje son:

1. Un espacio de estados
2. Un espacio de acciones disponibles
3. Un funcion de transicion

In [6]:
type RLEnv # reinforcement learning environment, should be immutable for efficiency
    state_space::Array{Any, 1} # 
    action_space::Array{Any, 1} # 
    trans_fun::Function # (state, action) -> (new_state, reward)
end

Por ejemplo, en nuestro problema el espacio de estados y de acciones es el mismo. Se puede estar en cualquier ciudad y una accion equivale a decidir a que ciudad irse. La funcion de transicion refleja el nuevo estado `s'` y el pago que `r` que un agente recibiria tras haber decidido una accion `a` estando en el estado `s`. Tanto `s'` como `r` pueden ser aleatorios. Veamos como seria una instanciacion para nuestro problema.

In [7]:
state_space = cities
action_space = cities
trans_fun(state, action) = action, - distArray[state, action] # la forma mas sencilla de definir funciones
europe_tour = RLEnv(state_space, action_space, trans_fun)

RLEnv(Any["Barcelona","Belgrade","Berlin","Brussels","Bucharest","Budapest","Copenhagen","Dublin","Hamburg","Istanbul"  …  "Moscow","Munich","Paris","Prague","Rome","Saint Petersburg","Sofia","Stockholm","Vienna","Warsaw"],Any["Barcelona","Belgrade","Berlin","Brussels","Bucharest","Budapest","Copenhagen","Dublin","Hamburg","Istanbul"  …  "Moscow","Munich","Paris","Prague","Rome","Saint Petersburg","Sofia","Stockholm","Vienna","Warsaw"],trans_fun)

Por ejemplo, supongamos que actualmente estamos en Vienna y quiseramos ir a Warsaw. Entonces para europe tour la *accion* ir a Vienna dado que estamos en el *estado* Barcelona esta data por

In [8]:
new_state, reward = europe_tour.trans_fun("Barcelona", "Paris")
println("Partiendo de Barcelona, se llego a la ciudad $new_state obteniendo un pago de $reward")

Partiendo de Barcelona, se llego a la ciudad Paris obteniendo un pago de -831.59


## Agentes que aprenden

Para poder diseñar un agente necesitamos conocer el estado de un agente, la politica con la que toma decisiones y el pago total acumulado que ha recibido. Un agente solo esta definido dentro de una tarea de aprendizaje.

In [9]:
type RLAgent
    policy::Function # state -> action
    state::Any # su estado actual
    total_reward::Real # el reward que ha acumulado haste el momento
end

Para que quede mas claro, vamos a crear un agente que en el problema del tour de europa se mueve al azar a cualquier otra ciudad partiendo de su estado actual. Vamos a suponer que el estado inicial es barcelona.

In [12]:
policy(state) = rand(europe_tour.action_space) # el agente mas tonto escoge una ciudad al azar sin depender del estado actual
state = "Barcelona"
total_reward = 0.0 # empezamos con cero reward acumulado
tour_agent = RLAgent(policy, state, total_reward)

RLAgent(policy,"Barcelona",0.0)

In [14]:
println("""La politica (aleatoria) que seguiria el agente tour_agent dado que actualmente se encuentra en
$(tour_agent.state) es ir a $(tour_agent.policy(tour_agent.state))""") # repetir este comando da distintos resultados

La politica (aleatoria) que seguiria el agente tour_agent dado que actualmente se encuentra en
Barcelona es ir a Belgrade


### Diseñando la interaccion

Para poder programar la interaccion entre un agente y su ambiente necesitamos programar *metodos*

Cuando un agente interactua con el ambiente deben ocurrir las siguientes cosas:

1. El agente escoge action data state segun su politica
2. El ambiente transiciona aleatoriamente, generando un nuevo estado new_state al gente y dando un reward en el proceso

Los metodos en julia son simplemente funciones cuyos argumentos son del tipo apropiado. A continuacion diseñamos un metodo de interaccion:

In [ ]:

function interact!(agent::RLAgent, env::RLEnv)
    # la convencion (opcional) de julia es incluir '!' al final de una funcion si modifica sus argumentos 
    new_state, reward = env.trans_fun(agent.state, agent.policy(agent.state))
    agent.total_reward += reward
    agent.state = new_state
    return new_state, reward # a veces es conveniente guardar
end